In [16]:
import numpy as np
import pandas as pd
import gensim.downloader as api
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import re
import random
from string import punctuation
from tqdm import tqdm

In [18]:
np.random.seed(42)
random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [2]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,topic,the concept of the topic,candidate,candidate masked,label,wikipedia article name,wikipedia url
0,We should limit executive compensation,executive compensation,A say on pay - a non-binding vote of the gener...,A say on pay - a non-binding vote of the gener...,0,Executive pay,https://en.wikipedia.org/wiki/Executive_pay
1,We should limit executive compensation,executive compensation,"A February 2009 report, published by the Insti...","A February 2009 report, published by the Insti...",1,Executive pay,https://en.wikipedia.org/wiki/Executive_pay
2,We should limit executive compensation,executive compensation,The Financial Crisis has had a relatively smal...,The Financial Crisis has had a relatively smal...,0,Executive pay,https://en.wikipedia.org/wiki/Executive_pay
3,We should limit executive compensation,executive compensation,"1990-1992 Lineberger Cancer Center, SPA person...","1990-1992 Lineberger Cancer Center, SPA person...",0,Edison Liu,https://en.wikipedia.org/wiki/Edison_Liu
4,We should limit executive compensation,executive compensation,Countering the public uproar over excessive ex...,Countering the public uproar over excessive TO...,0,Jack Welch,https://en.wikipedia.org/wiki/Jack_Welch


In [3]:
word_re = re.compile(r"\b[a-z]{2,}\b")

def tokenize(text):
    processed_text = "".join(ch for ch in text.lower() if ch not in punctuation)
    processed_text = processed_text.replace("\n", " ")
    return word_re.findall(processed_text)

In [4]:
all_train = train_df['candidate'].apply(tokenize)
test = test_df['candidate'].apply(tokenize)
all_train

0       [say, on, pay, nonbinding, vote, of, the, gene...
1       [february, report, published, by, the, institu...
2       [the, financial, crisis, has, had, relatively,...
3       [lineberger, cancer, center, spa, personnel, c...
4       [countering, the, public, uproar, over, excess...
                              ...                        
4060    [new, zealand, prime, minister, john, key, sai...
4061    [the, international, monarchist, league, found...
4062    [on, march, three, shiite, groups, formed, the...
4063    [the, gulf, states, were, especially, inclined...
4064    [in, may, poll, by, angus, reid, found, that, ...
Name: candidate, Length: 4065, dtype: object

In [5]:
embed_size = 300
wv = api.load(f'glove-wiki-gigaword-{embed_size}')

In [6]:
vocab = pd.Series(wv.key_to_index) + 1
encoded_train = all_train.apply(lambda x: [vocab.get(w, 0) for w in x])
encoded_test = test.apply(lambda x: [vocab.get(w, 0) for w in x])
encoded_train

0       [204, 14, 643, 27374, 539, 4, 1, 217, 287, 5, ...
1       [618, 256, 736, 22, 1, 1065, 11, 528, 1380, 21...
2       [1, 396, 722, 32, 41, 2224, 358, 1051, 1262, 1...
3              [249139, 1648, 314, 7213, 2076, 3532, 447]
4       [22804, 1, 199, 14954, 75, 6341, 617, 643, 145...
                              ...                        
4060    [51, 1273, 336, 142, 280, 639, 17, 19, 1055, 1...
4061    [1, 147, 47401, 293, 1298, 7, 43, 32, 52, 192,...
4062    [14, 305, 88, 2582, 504, 1348, 1, 1127, 11, 23...
4063    [1, 1666, 113, 36, 859, 11730, 5, 1656, 30, 18...
4064    [7, 108, 1888, 22, 16293, 5700, 239, 13, 57, 7...
Name: candidate, Length: 4065, dtype: object

In [7]:
encoded_labels = train_df['label'].to_numpy()
encoded_labels_test = test_df['label'].to_numpy()
encoded_labels

array([0, 1, 0, ..., 1, 1, 0])

In [8]:
def pad_data(encoded_data, seq_length):
    data = encoded_data[:seq_length]
    padded_data = np.zeros(seq_length, dtype=int)
    padded_data[-len(data):] = data
    return padded_data

padded_train = np.stack(encoded_train.apply(pad_data, seq_length=81))
padded_test = np.stack(encoded_test.apply(pad_data, seq_length=81))
padded_train

array([[    0,     0,     0, ...,   224,     4,   253],
       [    0,     0,     0, ...,     6,  3568, 19812],
       [    0,     0,     0, ...,    14,   617,   643],
       ...,
       [    0,     0,     0, ...,   591,   875, 29127],
       [    0,     0,     0, ...,     4,    79, 29127],
       [    0,     0,     0, ...,  4828,   915,   101]])

In [9]:
train_ratio = 0.85
total = padded_train.shape[0]
train_cutoff = int(total * train_ratio)

train_x, train_y = torch.from_numpy(padded_train[:train_cutoff]), torch.from_numpy(encoded_labels[:train_cutoff])
valid_x, valid_y = torch.from_numpy(padded_train[train_cutoff:]), torch.from_numpy(encoded_labels[train_cutoff:])

test_x, test_y = torch.from_numpy(padded_test), torch.from_numpy(encoded_labels_test)

from torch.utils.data import TensorDataset, DataLoader

train_data = TensorDataset(train_x, train_y)
valid_data = TensorDataset(valid_x, valid_y)
test_data = TensorDataset(test_x, test_y)

BATCH_SIZE = 32
train_loader = DataLoader(train_data, batch_size = BATCH_SIZE, shuffle = True)
valid_loader = DataLoader(valid_data, batch_size = BATCH_SIZE, shuffle = True)
test_loader = DataLoader(test_data, batch_size = BATCH_SIZE, shuffle = True)

In [20]:
class SentimentLSTM(nn.Module):

    def __init__(self, n_vocab, n_embed, n_hidden, n_output, n_layers, attention_size, drop_p = 0.85):
        super().__init__()
        # params: "n_" means dimension

        self.n_vocab = n_vocab
        self.n_layers = n_layers
        self.n_hidden = n_hidden

        self.embedding = nn.Embedding.from_pretrained(wv_torch_with_pad)
        self.lstm = nn.LSTM(n_embed, n_hidden, n_layers, bidirectional=True, batch_first = True, dropout = drop_p)
        self.fc = nn.Linear(n_hidden*2, n_output)

        self.attention = nn.Linear(n_hidden * 2, attention_size)
        self.attention_combine = nn.Linear(attention_size, 1, bias=False)

        self.dropout = nn.Dropout(drop_p)
        self.sigmoid = nn.Sigmoid()
        self.__init_linear()


    def forward (self, input_words, hidden_state=None):
        if hidden_state is None:
            h, c = self.init_hidden(input_words.shape[0])
        else:
            h, c = hidden_state
        x = self.embedding(input_words)
        lstm_out, (h, c) = self.lstm(x, (h, c))

        # Attention mechanism
        attention_weights = torch.tanh(self.attention(lstm_out))  # Get attention weights
        attention_weights = F.softmax(self.attention_combine(attention_weights), dim=1)
        context_vector = torch.sum(attention_weights * lstm_out, dim=1)

        # Classification
        fc_out = self.fc(context_vector)
        sigmoid_out = self.sigmoid(fc_out)

        return sigmoid_out, (h, c)


    def __init_linear(self):
        self.fc.weight.data.normal_(0.0, 1/np.sqrt(self.n_hidden * 2))
        self.fc.bias.data.fill_(0)


    def init_hidden(self, batch_size):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        weights = next(self.parameters()).data
        h = (weights.new(self.n_layers * 2, batch_size, self.n_hidden).zero_().to(device),
             weights.new(self.n_layers * 2, batch_size, self.n_hidden).zero_().to(device))

        return h

In [21]:
# torch.cat
wv_torch = torch.from_numpy(wv.vectors)
pad = torch.zeros((1, wv_torch.shape[1]))
wv_torch_with_pad = torch.cat((pad, wv_torch), dim=0)

In [31]:
n_vocab = wv_torch_with_pad.shape[0]
n_embed = wv_torch_with_pad.shape[1]
n_hidden = 128
n_output = 1
n_layers = 2
attention_size = 100

net = SentimentLSTM(n_vocab, n_embed, n_hidden, n_output, n_layers, attention_size)

criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters(), lr = 0.001, weight_decay=1e-5)

In [32]:
print_every = 50
step = 0
n_epochs = 12
clip = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)
net.train()

val_losses = []
val_accuracy = []
for epoch in tqdm(range(n_epochs)):
    for inputs, labels in tqdm(train_loader):
        step += 1
        inputs, labels = inputs.to(device), labels.to(device)

        output, t = net(inputs)
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()
        optimizer.zero_grad()

        if (step % print_every) == 0:
            ######################
            ##### VALIDATION #####
            ######################
            net.eval()
            valid_losses = []
            val_accuracy = []
            v_h = net.init_hidden(BATCH_SIZE)

            for v_inputs, v_labels in valid_loader:
                v_inputs, v_labels = inputs.to(device), labels.to(device)

                v_output, _ = net(v_inputs)
                v_loss = criterion(v_output.squeeze(), v_labels.float())
                valid_losses.append(v_loss.item())

                predictions = torch.round(torch.sigmoid(v_output.squeeze()))

            print("Epoch: {}/{}".format((epoch+1), n_epochs),
                  "Step: {}".format(step),
                  "Training Loss: {:.4f}".format(loss.item()),
                  "Validation Loss: {:.4f}".format(np.mean(valid_losses)))
            net.train()

 47%|███████████████████▊                      | 51/108 [00:04<00:09,  5.80it/s]

Epoch: 1/12 Step: 50 Training Loss: 0.5553 Validation Loss: 0.5419



 95%|███████████████████████████████████████  | 103/108 [00:08<00:00,  7.26it/s]

Epoch: 1/12 Step: 100 Training Loss: 0.5241 Validation Loss: 0.5124



 41%|█████████████████                         | 44/108 [00:03<00:08,  7.40it/s]

...


 13%|█████▍                                    | 14/108 [00:01<00:13,  7.19it/s]

Epoch: 12/12 Step: 1200 Training Loss: 0.0153 Validation Loss: 0.0125



 59%|████████████████████████▉                 | 64/108 [00:05<00:05,  7.42it/s]

Epoch: 12/12 Step: 1250 Training Loss: 0.0047 Validation Loss: 0.0029



100%|███████████████████████████████████████████| 12/12 [01:44<00:00,  8.71s/it]


In [33]:
net.eval()
test_losses = []
num_correct = 0
test_h = net.init_hidden(BATCH_SIZE)
net.to(device)
for inputs, labels in test_loader:
    inputs, labels = inputs.to(device), labels.to(device)
    test_output, _ = net(inputs)
    loss = criterion(test_output, labels.float().unsqueeze(dim=1))
    test_losses.append(loss.item())

    preds = torch.round(test_output.squeeze())
    correct_tensor = preds.eq(labels.float().view_as(preds))
    correct = np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)

print("Test Loss: {:.4f}".format(np.mean(test_losses)))
print("Test Accuracy: {:.2f}".format(num_correct / len(test_loader.dataset)))

Test Loss: 1.2259
Test Accuracy: 0.70
